<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Communicate_Data_Findings/Time_Series_Store_Sales_WorkOut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Store Sales - Time Series Forecasting
**Use machine learning to predict grocery sales**

[Kaggle Competition](https://www.kaggle.com/c/store-sales-time-series-forecasting/overview)

In [2]:
import os
# Set the environment variables
os.environ['KAGGLE_USERNAME'] = "lubomrstraka"
os.environ['KAGGLE_KEY'] = "c7347462ef834e6645ce238c2f2fa561"

# Upgrade kaggle library
os.system("pip install kaggle --upgrade --quiet")

# Download datasets
os.system("kaggle competitions download -c store-sales-time-series-forecasting --quiet")

  0% 0.00/135k [00:00<?, ?B/s]
100% 135k/135k [00:00<00:00, 44.1MB/s]
  0% 0.00/21.8k [00:00<?, ?B/s]
100% 21.8k/21.8k [00:00<00:00, 21.7MB/s]
  0% 0.00/334k [00:00<?, ?B/s]
100% 334k/334k [00:00<00:00, 46.0MB/s]
  0% 0.00/360k [00:00<?, ?B/s]
100% 360k/360k [00:00<00:00, 50.8MB/s]
  0% 0.00/20.1k [00:00<?, ?B/s]
100% 20.1k/20.1k [00:00<00:00, 22.6MB/s]
  0% 0.00/1.35k [00:00<?, ?B/s]
100% 1.35k/1.35k [00:00<00:00, 1.37MB/s]
 58% 13.0M/22.5M [00:00<00:00, 132MB/s]
100% 22.5M/22.5M [00:00<00:00, 143MB/s]


In [3]:
import zipfile
# Extract train.csv
with zipfile.ZipFile('train.csv.zip') as archive:
  archive.extractall()

In [5]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
dtype = {'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'uint64'}
store_sales = pd.read_csv('train.csv', dtype=dtype, parse_dates=['date'], infer_datetime_format=True)
store_sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    category      
 3   family       category      
 4   sales        float32       
 5   onpromotion  uint64        
dtypes: category(2), datetime64[ns](1), float32(1), int64(1), uint64(1)
memory usage: 85.9 MB


**A Data Set Description**

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | id | 3000888 | 3000888 | int64 | |
| 1 | date | 3000888 | 1684 | datetime64[ns] | |
| 2 | store_nbr | 3000888 | 54 | category | Identifies the store at which the products are sold. |
| 3 | family | 3000888 | 33 | category | Identifies the type of product sold. |
| 4 | sales | 3000888 | 379610 | float32 | Gives the total sales for a product family at a particular store at a given date. <br/>Fractional values are possible since products can be sold in fractional units <br/>(1.5 kg of cheese, for instance, as opposed to 1 bag of chips). |
| 5 | onpromotion | 3000888 | 991 | uint64 | Gives the total number of items in a product family that were being promoted at a store at a given date.|

Memory usage of `store_sales`:
- 137.4+ MB before applying the dict *dtype*
- 85.9+ MB after applying the dict *dtype*

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
